<a href="https://colab.research.google.com/github/njucs/notebook/blob/master/HuggingfaceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **简介** ###
目前各种Pretraining的Transformer模型层出不穷，虽然这些模型都有开源代码，但是它们的实现各不相同，我们在对比不同模型时也会很麻烦。Huggingface Transformer能够帮我们跟踪流行的新模型，并且提供统一的代码风格来使用BERT、XLNet和GPT等等各种不同的模型。而且它有一个模型仓库，所有常见的预训练模型和不同任务上fine-tuning的模型都可以在这里方便的下载。到目前为止，transformers 提供了超过100种语言的，32种预训练语言模型，简单，强大，高性能。

### **安装** ###

In [2]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 8.6 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
     |████████████████████████████████| 895 kB 73.9 MB/s 
     |████████████████████████████████| 3.3 MB 47.1 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


###**主要概念**###
1. 只有configuration，models和tokenizer三个主要类。
   - 诸如BertModel的模型(Model)类，包括30+PyTorch模型(torch.nn.Module)和对应的TensorFlow模型(tf.keras.Model)。模型列表可以参考https://huggingface.co/models
   - 诸如BertConfig的配置(Config)类，它保存了模型的相关(超)参数。我们通常不需要自己来构造它。如果我们不需要进行模型的修改，那么创建模型时会自动使用对于的配置
   - 诸如BertTokenizer的Tokenizer类，它保存了词典等信息并且实现了把字符串变成ID序列的功能。
   - 所有这三类对象都可以使用from_pretrained()函数自动通过名字或者目录进行构造，也可以使用save_pretrained()函数保存。
2. 所有的模型都可以通过统一的from_pretrained()函数来实现加载，transformers会处理下载、缓存和其它所有加载模型相关的细节。

###**使用**###

In [ ]:
# 使用 pipeline

'''
使用预训练模型最简单的方法就是使用pipeline函数，它支持如下的任务：
- 情感分析(Sentiment analysis)：一段文本是正面还是负面的情感倾向
- 文本生成(Text generation)：给定一段文本，让模型补充后面的内容
- 命名实体识别(Name entity recognition)：识别文字中出现的人名地名的命名实体
- 问答(Question answering)：给定一段文本以及针对它的一个问题，从文本中抽取答案
- 填词(Filling masked text)：把一段文字的某些部分mask住，然后让模型填空
- 摘要(Summarization)：根据一段长文本中生成简短的摘要
- 翻译(Translation)：把一种语言的文字翻译成另一种语言
- 特征提取(Feature extraction)：把一段文字用一个向量来表示
'''

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 除了通过名字来制定 model 参数，我们也可以传给 model 一个包含模型的目录的路径，也可以传递一个模型对象。
# 如果我们想传递模型对象，那么也需要传入 tokenizer。
# 我们需要两个类，一个是 AutoTokenizer，我们将使用它来下载和加载与模型匹配的 Tokenizer。
# 另一个是 AutoModelForSequenceClassification。
# 这两个 AutoXXX 类会根据加载的模型自动选择 Tokenizer 和 Model，如果我们提前知道了，也可以直接用对应的模型和 Tokenizer 进行 from_pretrained 调用
# 注意：模型类是与任务相关的，我们这里是情感分类的分类任务，所以是AutoModelForSequenceClassification。

# classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

results = classifier(["We are very happy to show you the 🤗 Transformers library.",
           "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
# 关于 Tokenizer 和 Model
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizer 的作用大致就是分词，然后把词变成的整数 ID，最终的目的是把一段文本变成 ID 的序列。
inputs = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(inputs)

# 也可以输入一个 batch
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
# pt_batch 仍然是一个 dict，input_ids 是一个 batch 的 ID 序列，
# 我们可以看到第二个字符串较短，所以它被 padding 成和第一个一样长。
# 如果某个句子的长度超过 max_length，也会被切掉多余的部分。
for key, value in pt_batch.items():
  print(f"{key}: {value.numpy().tolist()}")

# Tokenizer 的处理结果可以输入给模型，对于 PyTorch 需要使用 ** 来展开参数
# Transformers 的所有输出都是 tuple， 默认返回 logits，如果需要概率，可以自己加 softmax
pt_outputs = pt_model(**pt_batch)

# 如果有输出分类对应的标签，那么也可以传入，这样它除了会计算 logits 还会计算 loss
# pt_outputs = pt_model(**pt_batch, labels = torch.tensor([1, 0]))

# 也可以返回所有的隐状态和 attention
# pt_outputs = pt_model(**pt_batch, output_hidden_states=True, output_attentions=True)
# all_hidden_states, all_attentions = pt_outputs[-2:]

pt_predictions = F.softmax(pt_outputs[0], dim=-1)
print(pt_predictions)

In [ ]:
# 存储和加载使用
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

# 还可以轻松的在 PyTorch 和 TensorFlow 之间切换
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = TFAutoModel.from_pretrained(save_directory, from_pt=True)

# 如果用 PyTorch 加载 TensorFlow 模型，则需要设置 from_tf = True：
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModel.from_pretrained(save_directory, from_tf=True)

###**自定义模型（调整超参数，不是定义全新模型）**###

In [ ]:
# 需要构造配置类
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification

# 如果修改了核心的超参数，那么就不能使用 from_pretrained 加载预训练的模型了，必须从头开始训练模型
# Tokenizer 一般还是可以复用的

# Case 1: 修改核心超参数，构造 Tokenizer 和模型对象
config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4*512) # 修改超参数
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased') # Tokenizer 还是可以复用
model = DistilBertForSequenceClassification(config) # model 不能用 from_pretrained 加载了，需要重新训练

# Case 2: 只改变最后一层，比如把一个两分类的模型改成 10 分类的模型
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=10) # 通过设置 num_labels 参数来实现对最后一层的修改
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

'''
class DistilBertForSequenceClassification(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, config.num_labels)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

        self.init_weights()
'''

###**模型输入**
虽然基于transformer的模型各不相同，但是可以把输入抽象成统一的格式。

In [3]:
# 输入 ID
# 虽然不同的 tokenizer 实现差异很大，但是它们的作用是相同的，即把一个句子变成 Token 的序列，不同的 Token 有不同的整数 ID
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence = "A Titan RTX has 24GB of VRAM"

# 把句子变成 Token 序列
tokenized_sequence = tokenizer.tokenize(sequence)
print(tokenized_sequence)

# 把句子变成 ID 序列
inputs = tokenizer(sequence)
print(inputs)
encoded_sequence = inputs["input_ids"]
print(encoded_sequence)

# ID 的序列比 Token 要多两个元素，这是 Tokenizer 会自动增加一些特殊的 Token，比如 CLS 和 SEP
# 用 decode 来把 ID 解码成 Token
decoded_sequence = tokenizer.decode(encoded_sequence)
print(decoded_sequence)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

['A', 'Titan', 'R', '##T', '##X', 'has', '24', '##GB', 'of', 'V', '##RA', '##M']
{'input_ids': [101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[101, 138, 18696, 155, 1942, 3190, 1144, 1572, 13745, 1104, 159, 9664, 2107, 102]
[CLS] A Titan RTX has 24GB of VRAM [SEP]
